In [ ]:
# Standard imports
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union, Any
from dataclasses import dataclass
from abc import ABC, abstractmethod
from enum import Enum

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import roc_auc_score

## Configuration & Base Classes

In [ ]:
# ============== DatasetConfig ==============
@dataclass
class DatasetConfig:
    training_col_names: List[str]
    target_col_name: str
    numerical_col_names: List[str]
    categorical_col_names: List[str]
    name: Optional[str] = None


# ============== Base Preprocessor ==============
class BasePreprocessor(ABC):
    name = "base-preprocessor"

    def __init__(self, config: DatasetConfig):
        self._fitted = False
        self.config = config

    @abstractmethod
    def fit(self, data: pd.DataFrame) -> "BasePreprocessor":
        pass

    @abstractmethod
    def transform(self, data: pd.DataFrame) -> pd.DataFrame:
        pass

    def fit_transform(self, data: pd.DataFrame) -> pd.DataFrame:
        return self.fit(data).transform(data)

    @property
    def fitted(self) -> bool:
        return self._fitted


# ============== Base Model ==============
T = Union[np.ndarray, pd.DataFrame]

class BaseKtoolsModel(ABC):
    def __init__(self) -> None:
        self._fitted = False
        self.model = None

    @abstractmethod
    def fit(
        self,
        X: T,
        y: T,
        validation_set: Optional[Tuple[T, T]] = None,
        weights: Optional[T] = None,
        val_weights: Optional[T] = None,
    ) -> "BaseKtoolsModel":
        pass

    @abstractmethod
    def predict(self, X: T) -> np.ndarray:
        pass

    @property
    def fitted(self) -> bool:
        return self._fitted

## Preprocessing Classes

In [ ]:
# ============== Categorical Frequency Encoder ==============
class CategoricalFrequencyEncoder(BasePreprocessor):
    freq_suffix = "_frequency_encoding"

    def __init__(self, config: DatasetConfig, encode_missing_value: int = 0):
        super().__init__(config)
        self.train_freq_mappings: Dict[str, Dict[int, float]] = {}
        self.encode_missing_value = encode_missing_value

    def fit(self, data: pd.DataFrame) -> "CategoricalFrequencyEncoder":
        for column in self.config.categorical_col_names:
            new_col_name = column + self.freq_suffix
            if new_col_name in self.config.training_col_names:
                raise ValueError("Frequency encoded column already exists")

            freq_map = (
                data[column]
                .value_counts(normalize=True)
                .to_dict()
            )
            self.train_freq_mappings[column] = freq_map
        self._fitted = True
        return self

    def transform(self, data: pd.DataFrame) -> pd.DataFrame:
        copy = data.copy()
        for column in self.config.categorical_col_names:
            new_col_name = column + self.freq_suffix
            freq_map = self.train_freq_mappings[column]
            copy[new_col_name] = (
                copy[column]
                .astype(str)
                .map(freq_map)
                .fillna(self.encode_missing_value)
                .astype(float)
            )
        return copy


# ============== Categorical Target Encoder ==============
class CategoricalTargetEncoder(BasePreprocessor):
    def __init__(
        self,
        config: DatasetConfig,
        random_state: int = 42,
        cv: int = 5,
        smooth: int = 15,
        shuffle: bool = True,
    ):
        super().__init__(config)
        self.target_encoder = TargetEncoder(
            random_state=random_state, cv=cv, smooth=smooth, shuffle=shuffle
        )

    def fit(self, data: pd.DataFrame) -> "CategoricalTargetEncoder":
        self.target_encoder.fit(
            data[self.config.categorical_col_names], data[self.config.target_col_name]
        )
        self._fitted = True
        return self

    def transform(self, data: pd.DataFrame) -> pd.DataFrame:
        copy = data.copy()
        copy[self.config.categorical_col_names] = self.target_encoder.transform(
            copy[self.config.categorical_col_names]
        ).astype("float32")
        return copy

    def fit_transform(self, data: pd.DataFrame) -> pd.DataFrame:
        copy = data.copy()
        copy[self.config.categorical_col_names] = self.target_encoder.fit_transform(
            copy[self.config.categorical_col_names], copy[self.config.target_col_name]
        ).astype("float32")
        self._fitted = True
        return copy


# ============== Preprocessing Pipeline ==============
class PreprocessingPipeline:
    def __init__(self, preprocessors: List[BasePreprocessor]) -> None:
        self.preprocessors = preprocessors

    def train_pipe(self, data: pd.DataFrame) -> pd.DataFrame:
        for preprocessor in self.preprocessors:
            data = preprocessor.fit_transform(data)
        return data

    def inference_pipe(self, data: pd.DataFrame) -> pd.DataFrame:
        for preprocessor in self.preprocessors:
            data = preprocessor.transform(data)
        return data

## Model Classes

In [ ]:
# ============== Helper Function ==============
def infer_task(y: Union[np.ndarray, pd.Series]) -> str:
    """
    Will infer binary, multiclass classification or regression based on the target values.
    """
    if isinstance(y, pd.Series):
        y = y.to_numpy()
    y = y.flatten()

    nuniques = np.unique(y).shape[0]
    has_floats = np.any(y % 1 != 0)

    if has_floats:
        print("Target contains float values. Inferring regression task.")
        return "regression"
    elif nuniques == 2:
        print("Target contains two unique values. Inferring binary classification task.")
        return "binary_classification"
    elif nuniques > 2:
        print("Target contains more than two unique values. Inferring multiclass classification task.")
        return "multiclass_classification"

    raise ValueError(
        "Unable to infer task type from target values. Is there only one target value?"
    )


# ============== Default Objectives ==============
class DefaultObjective(Enum):
    regression = "regression"
    binary_classification = "binary"
    multiclass_classification = "multiclass"


# ============== LightGBM Model ==============
class LGBMModel(BaseKtoolsModel):
    def __init__(
        self,
        num_boost_round: int = 100,
        early_stopping_rounds: Union[int, None] = 20,
        random_state: int = 129,
        verbose: int = -1,
        n_jobs: int = 1,
        callbacks: List[Any] = [],
        **lgb_param_grid,
    ) -> None:
        super().__init__()
        self._num_boost_round = num_boost_round
        self._verbose = verbose
        self._n_jobs = n_jobs
        self._callbacks = callbacks
        self.early_stopping_rounds = early_stopping_rounds

        self._lgb_param_grid = {
            "verbose": verbose,
            "random_state": random_state,
            "n_jobs": n_jobs,
            **lgb_param_grid,
        }

    def fit(
        self,
        X: T,
        y: T,
        validation_set: Optional[Tuple[T, T]] = None,
        weights: Optional[T] = None,
        val_weights: Optional[T] = None
    ) -> "LGBMModel":
        if "objective" not in self._lgb_param_grid:
            task_id = infer_task(y)
            self._lgb_param_grid["objective"] = DefaultObjective[task_id].value
            if task_id == "multiclass_classification":
                self._lgb_param_grid["num_class"] = np.unique(y).shape[0]

        train_data = lgb.Dataset(X, label=y, weight=weights)
        eval_sets = [train_data]
        eval_names = ["train"]
        if validation_set is not None:
            X_val, y_val = validation_set
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data, weight=val_weights)
            eval_sets += [val_data]
            eval_names += ["valid"]
            self._lgb_param_grid["early_stopping_rounds"] = self.early_stopping_rounds

        train_params = {
            "params": self._lgb_param_grid,
            "train_set": train_data,
            "num_boost_round": self._num_boost_round,
            "valid_sets": eval_sets,
            "valid_names": eval_names,
            "callbacks": self._callbacks,
        }

        self.model = lgb.train(**train_params)
        self._fitted = True
        return self

    def predict(self, X: T) -> np.ndarray:
        y_pred = self.model.predict(X)
        return y_pred


from enum import Enum
from typing import *
import numpy as np
import pandas as pd
import xgboost as xgb


T = Union[np.ndarray, pd.DataFrame]


class DefaultObjective(Enum):
    regression = "reg:squarederror"
    binary_classification = "binary:logistic"
    multiclass_classification = "multi:softprob"


class XGBoostModel(BaseKtoolsModel):
    def __init__(
        self,
        eval_verbosity: bool = False,
        num_boost_round: int = 100,
        early_stopping_rounds: Union[int, None] = 20,
        random_state: int = 129,
        verbosity: int = 0,
        n_jobs: int = 1,
        **xgb_param_grid,
    ) -> None:
        super().__init__()
        self._eval_verbosity = eval_verbosity
        self._num_boost_round = num_boost_round
        self._verbosity = verbosity
        self._n_jobs = n_jobs
        self._early_stopping_rounds = early_stopping_rounds

        self._xgb_param_grid = {
            "verbosity": verbosity,
            "random_state": random_state,
            "n_jobs": n_jobs,
            **xgb_param_grid,
        }

    def fit(
        self,
        X: T,
        y: T,
        validation_set: Optional[Tuple[T, T]] = None,
        weights: Optional[T] = None,
        val_weights: Optional[T] = None
    ) -> "XGBoostModel":
        train_params = {}
        if "objective" not in self._xgb_param_grid:
            task_id = infer_task(y)
            self._xgb_param_grid["objective"] = DefaultObjective[task_id].value
            if task_id == "multiclass_classification":
                self._xgb_param_grid["num_class"] = np.unique(y).shape[0]

        train_data = xgb.DMatrix(X, label=y, enable_categorical=True, weight=weights)
        eval_data = [(train_data, "train")]
        if validation_set is not None:
            X_val, y_val = validation_set
            valid_data = xgb.DMatrix(X_val, label=y_val, enable_categorical=True, weight=val_weights)
            eval_data += [(valid_data, "eval")]
            train_params["early_stopping_rounds"] = self._early_stopping_rounds

        train_params = {
            "params": self._xgb_param_grid,
            "dtrain": train_data,
            "evals": eval_data,
            "num_boost_round": self._num_boost_round,
            "verbose_eval": self._eval_verbosity,
            **train_params,
        }

        self.model = xgb.train(**train_params)
        self._fitted = True
        return self

    def predict(self, X: T) -> np.ndarray:
        test_data = xgb.DMatrix(X, enable_categorical=True)
        y_pred = self.model.predict(test_data)
        return y_pred



# ============== Model Pipeline ==============
class ModelPipeline:
    def __init__(
        self,
        model: BaseKtoolsModel,
        config: DatasetConfig,
        preprocessor: PreprocessingPipeline = None,
    ) -> None:
        self.model = model
        self.config = config
        self.preprocessor = preprocessor if preprocessor else PreprocessingPipeline([])

    def fit(
        self,
        train_data: pd.DataFrame,
        validation_data: Optional[pd.DataFrame] = None,
        weights: Optional[Union[pd.Series, np.ndarray]] = None,
    ) -> "ModelPipeline":
        train_data = self.preprocessor.train_pipe(train_data)
        X_train = train_data.drop(columns=[self.config.target_col_name])
        y_train = train_data[self.config.target_col_name]

        if validation_data is not None:
            validation_data = self.preprocessor.inference_pipe(validation_data)
            X_valid = validation_data.drop(columns=[self.config.target_col_name])
            y_valid = validation_data[self.config.target_col_name]
            validation_data = (X_valid, y_valid)

        self.model.fit(
            X=X_train, y=y_train, validation_set=validation_data, weights=weights
        )
        return self

    def predict(self, data: pd.DataFrame) -> np.ndarray:
        data = self.preprocessor.inference_pipe(data)
        X_test = data #[self.config.training_col_names]
        return self.model.predict(X_test)

In [ ]:
# ============== CatBoost Model ==============
import catboost as cat
from catboost import Pool

class CatBoostModel(BaseKtoolsModel):
    """CatBoost model wrapper"""
    class DefaultObjective(Enum):
        regression = "RMSE"
        binary_classification = "Logloss"
        multiclass_classification = "MultiClass"

    def __init__(
        self,
        num_boost_round: int = 100,
        early_stopping_rounds: Optional[int] = 20,
        random_state: int = 129,
        verbose: bool = False,
        allow_writing_files: bool = False,
        **catboost_params,
    ) -> None:
        super().__init__()
        self.model: Union[cat.CatBoost, None] = None
        self._task: str = ""
        self._num_boost_round = num_boost_round
        self._verbose = verbose
        self._allow_writing_files = allow_writing_files
        self._early_stopping_rounds = early_stopping_rounds

        self._catboost_params = {
            "random_seed": random_state,
            "verbose": verbose,
            "allow_writing_files": allow_writing_files,
            **catboost_params,
        }

    def fit(
        self,
        X: T,
        y: T,
        validation_set: Optional[Tuple[T, T]] = None,
        weights: Optional[T] = None,
        val_weights: Optional[T] = None,
    ) -> "CatBoostModel":
        task_id = infer_task(y)
        self._task = task_id
        if "loss_function" not in self._catboost_params:
            self._catboost_params["loss_function"] = self.DefaultObjective[task_id].value

        self.cat_col_names = (
            [col for col in X.columns if X[col].dtype == "category"]
            if isinstance(X, pd.DataFrame)
            else []
        )
        train_params: Dict[Any, Any] = {"eval_set": None}
        train_pool = Pool(
            data=X, label=y, cat_features=self.cat_col_names, weight=weights
        )
        if validation_set is not None:
            X_val, y_val = validation_set
            train_params["eval_set"] = Pool(
                data=X_val, label=y_val, cat_features=self.cat_col_names, weight=val_weights
            )
            train_params["early_stopping_rounds"] = self._early_stopping_rounds

        train_params = {
            "params": self._catboost_params,
            "dtrain": train_pool,
            "num_boost_round": self._num_boost_round,
            **train_params,
        }
        self.model = cat.train(**train_params)
        self._fitted = True
        return self

    def predict(self, X: T) -> np.ndarray:
        if self.model is None:
            raise ValueError("Model is not fitted yet. Please call 'fit' first.")
        test_pool = Pool(data=X, cat_features=self.cat_col_names)
        if self._task == "binary_classification":
            y_pred = self.model.predict(test_pool, prediction_type="Probability")[:, 1]
        elif self._task == "multiclass_classification":
            y_pred = self.model.predict(test_pool, prediction_type="Probability")
        else:
            y_pred = self.model.predict(test_pool)
        return y_pred

## CV Tunable Function & Optuna Optimizer

In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.trial import Trial
from copy import deepcopy
import yaml

NestedDict = dict[str, "NestedDict | Any"]
TrialSampler = Callable[[Trial], Any]


def load_optuna_grid(
    path: str,
    model_type: str,
    extra_samplers: Dict[str, TrialSampler] | None = None,
) -> Callable[[Trial], Dict[str, Any]]:
    """
    Load an Optuna parameter grid from a YAML file.
    """
    with open(path, "r") as f:
        param_grid: NestedDict = yaml.safe_load(f)
    param_grid = param_grid.get(model_type, {})
    if len(param_grid) == 0:
        raise ValueError(f"No parameter grid found for model type: {model_type}")

    def param_grid_getter(trial: Trial) -> Dict[str, Any]:
        unpacked = {}
        for param_name, param_info in param_grid.items():
            dtype = param_info.get("type")
            if dtype == "int":
                unpacked[param_name] = trial.suggest_int(
                    param_name, param_info["low"], param_info["high"],
                )
            elif dtype == "float":
                unpacked[param_name] = trial.suggest_float(
                    param_name, param_info["low"], param_info["high"],
                    log=param_info.get("log", False),
                )
            elif dtype == "categorical":
                unpacked[param_name] = trial.suggest_categorical(
                    param_name, param_info["choices"],
                )
            elif dtype == "fixed":
                unpacked[param_name] = param_info["value"]
            else:
                raise ValueError(f"Unsupported parameter type: {dtype}")

        if extra_samplers:
            for param_name, sampler in extra_samplers.items():
                unpacked[param_name] = sampler(trial)

        return unpacked

    return param_grid_getter


def cv_tunable_func(
    train_data: pd.DataFrame,
    orig_data: pd.DataFrame,
    test_data: pd.DataFrame,
    categories_of_interest: pd.Series,
    config: DatasetConfig,
    preprocessors: List[BasePreprocessor],
    model_class: type,
    weight_map: Dict[int, float],
    n_splits: int = 5,
    random_state: int = 42,
    **model_params,
) -> float:
    """
    Cross-validation function to be tuned by Optuna.
    
    Follows the cv-experimentation pattern:
    - Stratified K-Fold on (target + data source)
    - Validation fold filtered to data == 0 only
    - Sample weights based on data source
    - Original data added to each training fold
    """
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    mean_score: float = 0.0
    
    for train_idx, val_idx in kfold.split(train_data, categories_of_interest):
        train_fold = train_data.iloc[train_idx].copy()
        val_fold = train_data.iloc[val_idx].copy()
        
        # Filter validation to data == 0 only
        subsetval_fold = val_fold[val_fold["data"] == 0]
        
        # Add original data to training fold
        train_fold = pd.concat([train_fold, orig_data], axis=0, ignore_index=True)
        
        # Sample weights from weight_map
        weights = train_fold["data"].map(weight_map).values
        
        # Fresh preprocessor pipeline for each fold
        preprocessor_pipeline = PreprocessingPipeline(
            preprocessors=[deepcopy(p) for p in preprocessors]
        )
        
        pipe = ModelPipeline(
            model=model_class(**model_params),
            config=config,
            preprocessor=preprocessor_pipeline,
        )
        
        pipe.fit(train_fold, validation_data=subsetval_fold, weights=weights)
        y_pred = pipe.predict(subsetval_fold.drop(columns=config.target_col_name))
        
        score = roc_auc_score(subsetval_fold[config.target_col_name], y_pred)
        mean_score += score / n_splits
    
    return mean_score


class OptunaHyperparameterOptimizer:
    """
    Hyperparameter optimizer using Optuna's TPE sampler.
    """

    def __init__(
        self,
        model_type: str,
        grid_yaml_path: str,
        extra_samplers: Dict[str, TrialSampler] | None = None,
        timeout: int = 3600,
        direction: str = "maximize",
        n_trials: int = 100,
        study_name: str = "ml_experiment",
        explore_fraction: float = 0.1,
        save_study: bool = False,
        load_if_exists: bool = True,
        catch_exceptions: Tuple[type, ...] = (),
        verbose: bool = False,
        random_state: int = 42,
    ) -> None:
        self._param_space_builder = load_optuna_grid(grid_yaml_path, model_type, extra_samplers=extra_samplers)
        self._timeout = timeout
        self._direction = direction
        self._n_trials = n_trials
        self._study_name = study_name
        self._explore_fraction = explore_fraction
        self._save_study = save_study
        self._load_if_exists = load_if_exists
        self._catch_exceptions = catch_exceptions
        self._verbose = verbose
        self._random_state = random_state
        self.study: optuna.Study | None = None

    def optimize(
        self,
        *args: Any,
        tunable_func: Callable[..., float],
    ) -> dict[str, Any]:
        """
        Run hyperparameter optimization.
        """
        if self._verbose:
            print("#" * 60)
            print("Starting Optuna Optimizer")
            print("#" * 60)

        sampler = TPESampler(
            n_startup_trials=int(self._n_trials * self._explore_fraction),
            seed=self._random_state,
        )

        storage_name = f"sqlite:///{self._study_name}.db" if self._save_study else None

        self.study = optuna.create_study(
            sampler=sampler,
            study_name=self._study_name,
            direction=self._direction,
            storage=storage_name,
            load_if_exists=self._load_if_exists,
        )

        def objective(trial: optuna.Trial) -> float:
            parameters = self._param_space_builder(trial)
            return tunable_func(*args, **parameters)

        self.study.optimize(
            objective,
            n_trials=self._n_trials,
            timeout=self._timeout,
            catch=self._catch_exceptions,
        )

        return self.study.best_params

## Data Loading & Preprocessing

This section implements the data preparation logic from the test file:
1. Load original, training, and test data
2. Assign data source labels (0=validation part of train, 1=train part, 2=original)
3. Create aggregated features from original data

In [ ]:
# ============== Configuration ==============
# For Kaggle, update these paths to match the competition data location
DATA_PATH = Path("/kaggle/input/playground-series-s5e12")
# DATA_PATH = Path("./data/diabetes_prediction/")  # Local path

TARGET = "diagnosed_diabetes"
SPLIT_ID = 678260  # Index to split training data into train/validation groups

In [ ]:
# ============== Load Data ==============
original_data = pd.read_csv("/kaggle/input/diabetes-health-indicators-dataset/diabetes_dataset.csv")
training_data = pd.read_csv(DATA_PATH / "train.csv", index_col=0)
test_data = pd.read_csv(DATA_PATH / "test.csv", index_col=0).assign(data=0)

print(f"Original data shape: {original_data.shape}")
print(f"Training data shape: {training_data.shape}")
print(f"Test data shape: {test_data.shape}")

In [ ]:
# ============== Assign Data Source Labels ==============
# data=0: validation portion of training data (after SPLIT_ID)
# data=1: training portion of training data (before SPLIT_ID)
# data=2: original external data

original_data = original_data.assign(data=2)
training_data = training_data.assign(data=0)
training_data.iloc[:SPLIT_ID, training_data.columns.get_loc('data')] = 1

print(f"Training data source distribution:")
print(training_data['data'].value_counts().sort_index())

In [ ]:
# ============== Create Aggregated Features from Original Data ==============
# Merge training, original and test data for feature engineering
train_orig_test = pd.concat(
    [training_data, original_data[training_data.columns], test_data], 
    axis=0, 
    ignore_index=True
)

training_cols = training_data.columns.drop(["data", TARGET]).tolist()
orig_target_mean = original_data[TARGET].mean()

print(f"Creating aggregated features from {len(training_cols)} columns...")

for c in training_cols:
    for aggr in ["mean", "count"]:
        col_name = f'{c}_org_{aggr}'
        tmp = (
            original_data.groupby(c)[TARGET]
            .agg(aggr)
            .rename(col_name)
            .reset_index()
        )
        
        train_orig_test = train_orig_test.merge(tmp, on=c, how='left')
        fill_val = orig_target_mean if aggr == 'mean' else 0
        train_orig_test[col_name] = train_orig_test[col_name].fillna(fill_val)

print(f"Combined data shape after feature engineering: {train_orig_test.shape}")

In [ ]:
# ============== Split Back into Train, Original, Test ==============
len_train = training_data.shape[0]
len_orig = original_data.shape[0]

train_data = train_orig_test.iloc[:len_train, :].copy()
orig_data = train_orig_test.iloc[len_train:len_train+len_orig, :].copy()
test_data = train_orig_test.iloc[len_train+len_orig:, :].copy().drop(columns=[TARGET])

print(f"Final train data shape: {train_data.shape}")
print(f"Final original data shape: {orig_data.shape}")
print(f"Final test data shape: {test_data.shape}")

## Dataset Configuration Setup

In [ ]:
# ============== Define Feature Categories ==============
all_features = train_data.columns.drop(TARGET).tolist()

# Categorical features include object/bool types + specific columns
categorical_features = (
    train_data.drop(columns=TARGET)
    .select_dtypes(include=['object', 'bool'])
    .columns.tolist() 
    + ['family_history_diabetes', 'hypertension_history', 'cardiovascular_history']
)

# Remove duplicates
categorical_features = list(set(categorical_features))

numerical_features = [col for col in all_features if col not in categorical_features]

print(f"Total features: {len(all_features)}")
print(f"Categorical features: {len(categorical_features)}")
print(f"Numerical features: {len(numerical_features)}")

In [ ]:
# ============== Create Dataset Config ==============
config = DatasetConfig(
    training_col_names=all_features,
    categorical_col_names=categorical_features,
    numerical_col_names=numerical_features,
    target_col_name=TARGET,
)

# Convert categorical columns to category dtype
train_data[categorical_features] = train_data[categorical_features].astype('category')
orig_data[categorical_features] = orig_data[categorical_features].astype('category')
test_data[categorical_features] = test_data[categorical_features].astype('category')

## Optuna Hyperparameter Tuning

Use the `OptunaHyperparameterOptimizer` with `cv_tunable_func` to tune XGBoost, LightGBM, or CatBoost.

In [ ]:
# ============== Setup for Optuna Tuning ==============
# Stratification categories
categories_of_interest = (
    train_data[TARGET].astype(str) + "_" + train_data["data"].astype(str)
)

# Sample weights based on data source
WEIGHT_MAP = {2: 8, 0: 16, 1: 1}

# Choose model to tune: XGBoostModel, LGBMModel, or CatBoostModel
MODEL_CLASS = XGBoostModel
GRID_YAML_PATH = "ktools/hyperopt/grids/xgb.yml"  # Update path as needed
MODEL_TYPE = "base"  # Key in YAML file

# Extra tunable parameters (e.g., sample weights)
extra_samplers = {
    "weight_data0": lambda t: t.suggest_float("weight_data0", 1.0, 20.0),
    "weight_data2": lambda t: t.suggest_float("weight_data2", 1.0, 15.0),
}

# Create optimizer
optimizer = OptunaHyperparameterOptimizer(
    model_type=MODEL_TYPE,
    grid_yaml_path=GRID_YAML_PATH,
    extra_samplers=extra_samplers,
    timeout=7200,  # 2 hours
    direction="maximize",
    n_trials=100,
    study_name="xgb_cv_tuning",
    explore_fraction=0.1,
    save_study=True,
    verbose=True,
    random_state=42,
)

print(f"Optimizer configured for {MODEL_CLASS.__name__}")

In [ ]:
# ============== Run Optuna Optimization ==============
# Uncomment to run (can take a long time!)

# def tunable_wrapper(
#     train_data, orig_data, test_data, categories_of_interest, config,
#     preprocessors, model_class, weight_map, n_splits, random_state,
#     weight_data0=16, weight_data2=8, **model_params
# ):
#     """Wrapper to inject tunable weights into weight_map."""
#     dynamic_weight_map = {0: weight_data0, 1: 1, 2: weight_data2}
#     return cv_tunable_func(
#         train_data=train_data,
#         orig_data=orig_data,
#         test_data=test_data,
#         categories_of_interest=categories_of_interest,
#         config=config,
#         preprocessors=preprocessors,
#         model_class=model_class,
#         weight_map=dynamic_weight_map,
#         n_splits=n_splits,
#         random_state=random_state,
#         **model_params,
#     )

# preprocessors = [
#     CategoricalFrequencyEncoder(config=config),
#     CategoricalTargetEncoder(config=config),
# ]

# best_params = optimizer.optimize(
#     train_data,
#     orig_data,
#     test_data,
#     categories_of_interest,
#     config,
#     preprocessors,
#     MODEL_CLASS,
#     WEIGHT_MAP,
#     5,  # n_splits
#     42,  # random_state
#     tunable_func=tunable_wrapper,
# )

# print(f"\nBest Score: {optimizer.study.best_value:.6f}")
# print(f"Best Params: {best_params}")

## Cross-Validation Experiment

Using Stratified K-Fold with stratification on both target and data source (from cell 8 of the original notebook).

In [ ]:
# ============== Stratification Categories ==============
# Combined stratification on target + data source
categories_of_interest = (
    train_data[TARGET].astype(str) + "_" + train_data["data"].astype(str)
)

print("Stratification category distribution:")
print(categories_of_interest.value_counts())

In [ ]:
DEVICE = "gpu"
SEED = 42

XGB_PARAMS = {
    'tree_method': 'hist',
    'objective': 'binary:logistic',
    'random_state': SEED,
    'eval_metric': 'auc',
    'booster': 'gbtree',
    'n_jobs': -1,
    'learning_rate': 0.01,
    "device": DEVICE,
    'lambda': 1.134330929497114,
    'alpha': 6.780537184218281,
    'colsample_bytree': 0.1007615968427798,
    'subsample': 0.7215917619002097,
    'max_depth': 4,
    'min_child_weight': 5,
    'num_boost_round' : 100_000,
    'early_stopping_rounds':200,
    # Note: enable_categorical is passed to DMatrix, not params in native API usually,
    # but kept here for consistency with modern XGBoost versions.
}

In [ ]:
# ============== CV Training Loop ==============
N_SPLITS = 5
RANDOM_STATE = 42

# Initialize arrays for predictions
train_oof_preds = np.empty(train_data.shape[0])
test_oof_preds = np.zeros(test_data.shape[0])

# Sample weights based on data source
# data=2 (original): weight 8
# data=0 (val portion): weight 16  
# data=1 (train portion): weight 1
WEIGHT_MAP = {2: 8, 0: 16, 1: 1}


kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
mean_score = 0.0
fold_scores = []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(train_data, categories_of_interest)):
    print(f"\n{'='*50}")
    print(f"Fold {fold_idx + 1}/{N_SPLITS}")
    print(f"{'='*50}")
    
    # Split data
    train_fold = train_data.iloc[train_idx].copy()
    val_fold = train_data.iloc[val_idx].copy()
    
    # Subset validation fold for scoring (only data=0, the "real" validation portion)
    subsetval_fold = val_fold[val_fold["data"] == 0]
    
    # Add original data to training fold
    train_fold = pd.concat([train_fold, orig_data], axis=0, ignore_index=True)
    
    print(f"Train fold size: {train_fold.shape[0]} (including {orig_data.shape[0]} original samples)")
    print(f"Validation fold size: {val_fold.shape[0]} (subset for scoring: {subsetval_fold.shape[0]})")
    
    # Calculate sample weights
    weights = train_fold["data"].map(WEIGHT_MAP).values
    
    # Use ModelPipeline with PreprocessingPipeline

    preprocessors = [
        CategoricalFrequencyEncoder(config=config),
        CategoricalTargetEncoder(config=config),
    ]
    
    pipe = ModelPipeline(
        model=XGBoostModel(**XGB_PARAMS),
        config=config,
        preprocessor=PreprocessingPipeline(preprocessors=preprocessors),
    )
    
    pipe.fit(train_fold, validation_data=subsetval_fold, weights=weights)
    
    # Make predictions
    y_pred = pipe.predict(subsetval_fold.drop(columns=TARGET))
    test_pred = pipe.predict(test_data)
    oof_pred = pipe.predict(val_fold.drop(columns=TARGET))
    
    # Calculate fold score on SUBSET validation (data=0 only)
    fold_score = roc_auc_score(subsetval_fold[TARGET], y_pred)
    fold_scores.append(fold_score)
    
    # Store OOF predictions for full validation fold
    train_oof_preds[val_idx] = oof_pred
    test_oof_preds += test_pred / N_SPLITS
    mean_score += fold_score / N_SPLITS
    
    print(f"Fold {fold_idx + 1} ROC AUC Score: {fold_score:.6f}")

print(f"\n{'='*50}")
print(f"CV Results")
print(f"{'='*50}")
print(f"Individual fold scores: {[f'{s:.6f}' for s in fold_scores]}")
print(f"Mean ROC AUC Score: {mean_score:.6f}")
print(f"Std ROC AUC Score: {np.std(fold_scores):.6f}")

## OOF Score & Submission

In [ ]:
# ============== Overall OOF Score ==============
overall_oof_score = roc_auc_score(train_data[TARGET], train_oof_preds)
print(f"Overall OOF ROC AUC Score: {overall_oof_score:.6f}")

In [ ]:
submission = pd.DataFrame({'id': test_data.index, TARGET: test_oof_preds})
submission.to_csv('submission.csv', index=False)

In [ ]:
# ============== Create Submission ==============
# For Kaggle submission
# sample_sub = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col=0)
# sample_sub[TARGET] = test_oof_preds
# sample_sub.to_csv("submission.csv")
# print("Submission saved!")

# Preview predictions
print(f"Test predictions stats:")
print(f"  Min: {test_oof_preds.min():.6f}")
print(f"  Max: {test_oof_preds.max():.6f}")
print(f"  Mean: {test_oof_preds.mean():.6f}")

In [ ]:
# ============== Save OOF and Test Predictions ==============
import uuid

guid = uuid.uuid4()
print(f"Experiment GUID: {guid}")

# Uncomment to save predictions
# save_path = DATA_PATH
# pd.DataFrame({f"{guid}": train_oof_preds}).to_csv(save_path / "oofs" / f"oof_preds_{guid}.csv")
# pd.DataFrame({f"{guid}": test_oof_preds}).to_csv(save_path / "test_preds" / f"test_preds_{guid}.csv")
# print(f"Predictions saved with GUID: {guid}")